In [18]:
from utility import cifar10
from utility.cifar10 import img_size
import tensorflow as tf
import sys

In [2]:
def prepare_train_data():
    cifar10.maybe_download_and_extract()
    images_train, cls_train, labels_train = cifar10.load_training_data()
    print("Size of:")
    print("- Training-set:\t\t{}".format(len(images_train)))   
    return images_train, cls_train, labels_train

In [3]:
def prepare_test_data():
    cifar10.maybe_download_and_extract()
    
    images_test, cls_test, labels_test = cifar10.load_test_data()
    print("Size of:")    
    print("- Test-set:\t\t{}".format(len(images_test)))
    return images_test, cls_test, labels_test

In [4]:
#Get Train Data
images_train, cls_train, labels_train = prepare_train_data()

Data has apparently already been downloaded and unpacked.
Loading data: D:\Users\amit.agarwal\PycharmProjects\Scikit_Learn_TensorFlow_Examples\Examples-TensorFlow\cifar-cnn\data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: D:\Users\amit.agarwal\PycharmProjects\Scikit_Learn_TensorFlow_Examples\Examples-TensorFlow\cifar-cnn\data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: D:\Users\amit.agarwal\PycharmProjects\Scikit_Learn_TensorFlow_Examples\Examples-TensorFlow\cifar-cnn\data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: D:\Users\amit.agarwal\PycharmProjects\Scikit_Learn_TensorFlow_Examples\Examples-TensorFlow\cifar-cnn\data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: D:\Users\amit.agarwal\PycharmProjects\Scikit_Learn_TensorFlow_Examples\Examples-TensorFlow\cifar-cnn\data/CIFAR-10/cifar-10-batches-py/data_batch_5
Size of:
- Training-set:		50000


In [5]:
#Get Test Data
#images_test, cls_test, labels_test=prepare_test_data()


In [6]:
images_train.shape
cifar10.load_class_names()

Loading data: D:\Users\amit.agarwal\PycharmProjects\Scikit_Learn_TensorFlow_Examples\Examples-TensorFlow\cifar-cnn\data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [7]:
def weight_init(shape,name):
    weight = tf.get_variable(name=name,dtype=tf.float32,shape=shape,initializer=tf.contrib.layers.variance_scaling_initializer())
    return weight

def bias_init(shape,name):
    bias = tf.get_variable(name=name,dtype=tf.float32,shape=shape,initializer=tf.contrib.layers.variance_scaling_initializer())
    return bias

In [28]:
def single_layer(X,Y_):
    weight1 = weight_init([3072, 10], name='weight1')
    bias1 = bias_init([10], name='bias1')
    XX = tf.reshape(X, [-1, 3072])
    activation = tf.matmul(XX,weight1) + bias1 
    Y = tf.nn.softmax(activation)
    loss = -tf.reduce_mean(Y_ * tf.log(Y+ 1e-15)) 
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return activation,loss,Y,accuracy

In [29]:
def single_layer_softmax():
    #Get Data
    #images_train, cls_train, labels_train = prepare_train_data()
    #images_test, cls_test, labels_test=prepare_test_data()
    print("Entering the function")
    # input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
    X = tf.placeholder(tf.float32,shape=[None,32,32,3])
    # correct answers will go here
    Y_ = tf.placeholder(tf.float32, [None, 10])

    n_epoch = 100
    batch_size = 100

    activation,loss,Y,accuracy = single_layer(X,Y_)
    # training, learning rate = 0.005
    train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(n_epoch):
            n_batches = images_train.shape[0] // batch_size
            # n_batches = n_batches_dimension.value
            idx = 0
            for iteration in range(n_batches):
                X_batch = images_train[idx:idx + batch_size]
                Y_batch = labels_train[idx:idx + batch_size]
                idx += batch_size
                print("\r{}%".format(100 * iteration // n_batches), end="") 
                sys.stdout.flush()
                sess.run(train_step, feed_dict={X: X_batch, Y_: Y_batch})
            if epoch % 10 == 0:                
                loss_train = loss.eval(feed_dict={X: X_batch, Y_: Y_batch})     
                print("\r{}".format(epoch), "Train MSE:", loss_train)
                 
            if epoch % 15 == 0:
                accuracy_train = accuracy.eval(feed_dict={X: X_batch, Y_: Y_batch})
                
                print("\r{}".format(epoch), "Train Accuracy:", accuracy_train) 
                
                


In [27]:
tf.reset_default_graph() 
single_layer_softmax()

Entering the function
0 Train MSE: 2692.9414
0 Train Accuracy: 0.16
10 Train MSE: 1679.0646
15 Train Accuracy: 0.35
20% Train MSE: 1539.165
30 Train MSE: 1707.9492
30 Train Accuracy: 0.4
40% Train MSE: 1721.1123
45 Train Accuracy: 0.35
50 Train MSE: 1545.7026
60 Train MSE: 1394.1929
60 Train Accuracy: 0.4
70 Train MSE: 1511.0764
75 Train Accuracy: 0.36
80 Train MSE: 1407.1412
90 Train MSE: 1632.7764
90 Train Accuracy: 0.35
99%